In [1]:
import sys
from awsglue.transforms import ResolveChoice,Relationalize
from awsglue.utils import getResolvedOptions
from awsglue.dynamicframe import DynamicFrame
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
## args = getResolvedOptions(sys.argv, ['JOB_NAME'])

## sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
## job.init(args['JOB_NAME'], args)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1602238379821_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
glue_db_name = 'nyphil_db'
glue_orig_table_name = 'ny_phil_perf_hist'
s3_temp_folder = 's3://bdb683-databucket-1uqqkb2fs03br/tmp/'
s3_target_path = 's3://bdb683-databucket-1uqqkb2fs03br/data-lake2/'
redshift_db_name = 'nyphil_db'
redshift_connection = 'ny_phil_redshift_conn'
redshift_schema = 'ny_phil'
root_table = 'ny_phil_perf_hist'
lvl_to_denorm = 99
num_output_files = 4
keep_table_prefix = False
target_repo = 'all'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# @type: DataSource
# @args: [database = glue_db_name, table_name = glue_orig_table_name, transformation_ctx = "datasource0"]
# @return: datasource0
# @inputs: []
datasource0 = glueContext.create_dynamic_frame.from_catalog(
    database=glue_db_name, table_name=glue_orig_table_name, transformation_ctx="datasource0")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
datasource0.printSchema()

In [4]:
resolvechoice2 = ResolveChoice.apply(
    frame=datasource0, choice="make_struct", transformation_ctx="resolvechoice2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
resolvechoice2.printSchema()

In [5]:
relationalize4 = Relationalize.apply(
    frame=resolvechoice2, staging_path=s3_temp_folder, name=root_table, transformation_ctx="relationalize4")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for df_name in relationalize4.keys():
    print(df_name)

In [ ]:
for df_name in relationalize4.keys():
    print(df_name)
    relationalize4.select(df_name).toDF().schema.names

In [ ]:
relationalize4.select('ny_phil_perf_hist_programs').toDF().schema.names

In [6]:
dynframes_list = {}
dataframes_list = {}
s3_target_path_list = {}
column_names_list = {}
table_names_list = []
datasink = {}

# variables needed to automatically join the tables
# lvl_to_denorm is set to 0 by default, this means no join will be processed
# set it to 99 to create a single denormalized table, or to an integer corrisponding
# to the nested levels you want to join (if there are less levels than requested a single table will be created)

join_lvl = 0
join_start_lvl = 0
table_tojoin = {}
denorm_df_list={}
# set the correct level to start the denormalization based on the input parameter

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# helper functions definition, this are basic snippet of codes used to simplify and standardize
# the name cleansing and writing to the target data stores:

# add a prefix to a name

def add_prefix(c, prefix):
    if prefix in c:
        r = c
    else:
        r = prefix+'_'+c
    return r

# check if column is a foreign key 

def is_foreign_key(name):
    return name[-3:]=="_fk"

# extract a table name from a foreign key (fk) attribute

def get_child_tbl_name(name):
    return name.replace("_fk", "")

# check if we want to denormalize the tables or not

def is_to_denormalize(lvl_to_denorm):
    return lvl_to_denorm>0

# cleans a name from the automatic code inserted strings to make the table and columns name more readable

def clean_name(name_type, tbl_name, col_name, reference_list, text_to_replace):
    if name_type == 'table':
        new_name = tbl_name.lower().replace(
            text_to_replace, "").replace(".", "_").replace("_val_", "_")
    elif name_type == 'column':
        new_name = col_name.lower().replace(
            ".val.", "_").replace(".", "_").replace("__", "_")
        if new_name in reference_list:
            new_name = new_name+"_fk"
        elif new_name == 'id':
            new_name = tbl_name+"_sk"
        elif new_name == 'index':
            new_name = "rownum"
        elif not keep_table_prefix:
            new_name = new_name.replace(tbl_name+"_", "")
    return new_name

# find if the table is to be joined and which level in the join hierarchy is at

def find_tbl_join_lvl(dict, tbl):
    for k in dict:
        if tbl in dict[k]:
            lvl=int(k)
            found=1
            break
        else:
            lvl=0
            found=0
    return {'join_lvl':lvl,'table_found':found}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
for df_name in relationalize4.keys():
    
    if (df_name == root_table):
        if len(relationalize4.select(root_table).toDF().schema.names) == 1:
            tbl_name = relationalize4.select(root_table).toDF().schema.names[0]
        else:
             tbl_name = root_table
        table_tojoin={'0': {tbl_name: {'join_to_tbls':[],'join_col':[]}}}
        table_names_list.insert(0,tbl_name)
    else:
        tbl_name = clean_name('table', df_name, '', [], root_table+"_")
        if tbl_name not in table_names_list:
            table_names_list.append(tbl_name)
    
            
    # create a list of S3 output bucket indexed by table name
    s3_target_path_list[tbl_name] = s3_target_path+tbl_name
    # create a list of dataframes indexed by table name
    dataframes_list[tbl_name] = relationalize4.select(df_name).toDF()
    # create a list of columns indexed by table name
    column_names_list[tbl_name] = dataframes_list[tbl_name].schema.names


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:

# write to target function, leverages glue native dynamic frame writer and standardize the parameter to pass,
# added to avoid to repeate the code multiple time and assure it is always using consistent parameters


def write_to_targets(tbl_name, dyn_frame, target_path, num_output_files, target_repo):

    dyn_frame = dyn_frame.coalesce(num_output_files)
    if target_repo == 's3' or target_repo == 'all':
        datasink[tbl_name] = glueContext.write_dynamic_frame.from_options(frame=dyn_frame, connection_type="s3",
                                                                          connection_options={
                                                                              "path": target_path},
                                                                          format="glueparquet",
                                                                          transformation_ctx="datasink['tbl_name']")
    if target_repo == 'redshift' or target_repo == 'all':
        db_tbl_name = redshift_schema+"."+tbl_name
        glueContext.write_dynamic_frame.from_jdbc_conf(frame=dyn_frame,
                                                       catalog_connection=redshift_connection,
                                                       connection_options={
                                                           "dbtable": db_tbl_name, "database": redshift_db_name},
                                                       redshift_tmp_dir=s3_temp_folder)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Loop on the list of tables and for each table on the attributes to standardize each column name

for tbl_name in table_names_list:
    
    # set the join_lvl for this table and if the table is in the list to be denormalized
    join_lvl = find_tbl_join_lvl(table_tojoin, tbl_name)['join_lvl']
    table_to_denorm=find_tbl_join_lvl(table_tojoin, tbl_name)['table_found']   
    for col_name in column_names_list[tbl_name]:
        
        # Clean column name and rename the dataframe column 
        new_col_name = clean_name('column', tbl_name, col_name, table_names_list, '')
        dataframes_list[tbl_name] = dataframes_list[tbl_name].withColumnRenamed(col_name, new_col_name).drop('rownum')
        
        # if the column is a foreing key and we have configured the job to run the normalization we will add the child table to the list of table to join 
        if is_foreign_key(new_col_name) and is_to_denormalize(lvl_to_denorm):
            
            # get the child table name and join level
            child_tbl_name= get_child_tbl_name(new_col_name)
            next_join_lvl= join_lvl+1
            
            # add the child table to the list of tables to join to for the current teable, and the foreign keys
            table_tojoin[str(join_lvl)][tbl_name]['join_to_tbls'].append(child_tbl_name)
            table_tojoin[str(join_lvl)][tbl_name]['join_col'].append(new_col_name)
            table_tojoin[str(join_lvl)][tbl_name]['has_child']=True
            
            # add the child table to the next level in the table_tojoin dictionary
            if len(table_tojoin) == next_join_lvl:
                table_tojoin[str(next_join_lvl)]={}
            table_tojoin[str(next_join_lvl)][child_tbl_name]={'join_to_tbls':[],'join_col':[],'has_child':False}
                
    # if the table is not in the list to be denormalized it does not have childs and depending the level of normalization requested it might be needed to write out anyway
    if table_to_denorm == 0 and is_to_denormalize(lvl_to_denorm):
        table_to_write_list.append(tbl_name)
        
    # convert to Dynamicframes in preparation for write
    dynframes_list[tbl_name] = DynamicFrame.fromDF(
        dataframes_list[tbl_name], glueContext, "dynframes_list[tbl_name]")
    
    # if no denormalizarion required write out the data
    #if not is_to_denormalize(lvl_to_denorm):
    #    write_to_targets(tbl, dynframes_list[tbl], s3_target_path_list[tbl],num_output_files)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# recurse on the list of table to be joined and join starting by the deepest level
def denormalize_table(table_tojoin,join_start_lvl, join_lvl,dataframes_list,denorm_df_list ):
    # set the next join level and start looping on table to be joined at curent level (it will be the left table in the join)
    next_join_level = join_start_lvl+1
    for left_table in table_tojoin[str(join_start_lvl)]:
        # last join level contains table with not further joins, so we check if we are already at the brfore last join level, if not we recurse to the next level 
        if next_join_level < join_lvl:
            denormalize_table(table_tojoin,next_join_level, join_lvl,dataframes_list,denorm_df_list )
        # if done with recursion execute the join, first set the left data frame for the join
        df_left=dataframes_list[left_table]
        
        # loop on all table listed in the join_to_table list for the left_table if the table has childs
        if table_tojoin[str(join_start_lvl)][left_table]['has_child']:
            for right_table in table_tojoin[str(join_start_lvl)][left_table]['join_to_tbls']:
            
                # look up the foreing key and set it as join column for left table
                join_col_index = table_tojoin[str(join_start_lvl)][left_table]['join_col'].index(right_table+"_fk")
                join_col_left = table_tojoin[str(join_start_lvl)][left_table]['join_col'][join_col_index]
            
                # set the join column for the right table replacing the suffix (remember we have named the column in a prebious step so we own the naming convention)
                join_col_right = join_col_left.replace("_fk","_sk")
            
                # if the right table had been already denormalized use that dataframe otherwise use the original dataframe
                if right_table in denorm_df_list:
                    df_right=denorm_df_list[right_table]
                else:
                    df_right=dataframes_list[right_table]
            
                # in case we decided to remove the table name prefix from the columns than check if there is a naming conflict and resolve it adding back the tablename prefix
                left_cols = df_left.schema.names
                right_cols = df_right.schema.names
                #right_cols.pop(right_cols.index("rownum"))
                for col in right_cols:
                    if col in left_cols:
                        df_right=df_right.withColumnRenamed(col, right_table+"_"+col)
                        right_cols.append(right_table+"_"+col)
                        right_cols.remove(col)
            
                #execute the join matching left and right join column names to avoid column duplication in the output dataframe
                df_left = df_left.join(df_right.withColumnRenamed(join_col_right, join_col_left), join_col_left, how='left_outer')
        
        #add the denormalized data frame and return it to the previous recursion level
        denorm_df_list[left_table]=df_left
            
    return denorm_df_list

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# if no denormalization needed we are done 
if not is_to_denormalize(lvl_to_denorm):
    print('Job completed Successfully')
else:
    if lvl_to_denorm > join_lvl:
        join_start_lvl = 0
    else:
        join_start_lvl = (join_lvl - lvl_to_denorm)
        print('start denormalizing table(s)')

    # start denormalization
    denorm_df_list=denormalize_table(table_tojoin,join_start_lvl,join_lvl,dataframes_list,denorm_df_list)
    
    # loop to write out the right dynamic frames
    for tbl in dynframes_list:
        # we need to write only the denormalized dynamic frames and any frame that has been skipped by the process
        table_to_be_joined_with_level = find_tbl_join_lvl(table_tojoin,tbl)
        join_lvl_pr = find_tbl_join_lvl(table_tojoin, tbl)['join_lvl']
        table_to_denorm_pr=find_tbl_join_lvl(table_tojoin, tbl)['table_found']
        
        if table_to_be_joined_with_level['join_lvl'] <= join_start_lvl:
            if table_to_be_joined_with_level['table_found'] == 1 and table_to_be_joined_with_level['join_lvl'] == join_start_lvl:
                # if a table has been normalized convert the dataframe to dynamicFrame 
                dynframes_list[tbl] = DynamicFrame.fromDF(denorm_df_list[tbl], glueContext, "dynframes_list[tbl]")
            #write_to_targets(tbl, dynframes_list[tbl], s3_target_path_list[tbl],num_output_files)
    

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- programs_works_fk: long
|-- programs_concerts_fk: long
|-- programs_sk: long
|-- id: string
|-- programid: string
|-- orchestra: string
|-- season: string
|-- eventtype: string
|-- location: string
|-- venue: string
|-- date: string
|-- time: string
|-- programs_works_soloists_fk: long
|-- programs_works_id: string
|-- composername: string
|-- conductorname: string
|-- worktitle_string: string
|-- worktitle_struct_: string
|-- worktitle_struct_em: string
|-- interval: string
|-- movement_string: string
|-- movement_struct_: string
|-- movement_struct_em: string
|-- soloistname: string
|-- soloistinstrument: string
|-- soloistroles: string